### Importing functions and classes

In [88]:
from IPython.display import HTML,clear_output,display
from ipywidgets.widgets import Button,Text,FloatSlider
from time import sleep
from copy import deepcopy

## Work done on the initial idea of the game

Some stuff for just looks

In [89]:
%%HTML
<style>
    td {
        color: white;
        margin: 1px;
        border-radius: 3px;
        text-align: center;
    }
    .board{
        border: 2px solid rgb(104, 90, 90);
        width: fit-content;
        background-color: white;
    }
    table{
        padding: 1px;
        border-collapse: separate;
        border-spacing: 1px;
    }
</style>

Making the interface for the game

In [90]:
M = [
    [[1,2],[1,2],[1,2]],
    [[1,2],None,[1,2]],
    [[1,2],[1,2],[1,2]]
]
X = None
T = [
    [X,2,4,X,5,1],
    [5,X,0,2,X,3],
    [1,3,X,4,0,X],
    [X,5,1,X,2,4],
    [2,X,3,5,X,0],
    [4,0,X,1,3,X]
]
def Top(x,y):
    return T[x][y]
def Bottom(x,y):
    return T[y][x]
def disp(M):
    colors = ["black","darkblue","darkgreen","orange","darkpink","gray"]
    S = "<div class='board'><table>"
    for row in M:
        S += "\n <tr>"
        for cell in row:
            if cell == None:
                S += f"\n <td bgcolor=transparent> </td>"
                continue
            x = Top(cell[0],cell[1])
            S += f"\n <td bgcolor={colors[x]}> &nbsp; <small>{cell[1]}</small> <br> <small>{cell[0]}</small> &nbsp; <big>{x}</big> &nbsp;&nbsp; <br> &nbsp;</td>"
        S += "\n </tr>"
    S += "</table></div>"
    display(HTML(S))
def L(M):
    for i in range(len(M)):
        for j in range(len(M[0])):
            cell = M[i][j]
            if cell == None and j>0:
                M[i][j],M[i][j-1]=M[i][j-1],M[i][j]
                cell = M[i][j]
                cell[0]=Bottom(cell[0],cell[1])
                return True
    return False
def R(M):
    for i in range(len(M)):
        for j in range(len(M[0])):
            cell = M[i][j]
            if cell == None and j<len(M[0])-1:
                M[i][j],M[i][j+1]=M[i][j+1],M[i][j]
                cell = M[i][j]
                cell[0]=Top(cell[0],cell[1])
                return True
    return False  
def U(M):
    for i in range(len(M)):
        for j in range(len(M[0])):
            cell = M[i][j]
            if cell == None and i>0:
                M[i-1][j],M[i][j]=M[i][j],M[i-1][j]
                cell = M[i][j]
                cell[1]=Bottom(cell[0],cell[1])
                return True
    return False
def D(M):
    for i in range(len(M)):
        for j in range(len(M[0])):
            cell = M[i][j]
            if cell == None and i<len(M)-1:
                M[i+1][j],M[i][j]=M[i][j],M[i+1][j]
                cell = M[i][j]
                cell[1]=Top(cell[0],cell[1])
                return True
    return False
Moves = {
    "U":U,
    "D":D,
    "L":L,
    "R":R,
    "u":U,
    "d":D,
    "l":L,
    "r":R,
}
def apply(s,M,dt=0):
    for c in s :
        Moves[c](M)
        if dt>0:
            clear_output(True)
            disp(M)
            sleep(dt)
but = Button()
but.description="Play"
tex = Text()
wait = FloatSlider()
wait.max=5
def play(event):
    s = tex.value
    dt = float(wait.value)
    print(dt)
    apply(s,M,dt)
    clear_output(True)
    disp(M)
    display(tex,wait,but)
but.on_click(play)
print("Type commands in the textbox, and press the button to play the animation. Toggle the slider to turn on and off the animation.")
disp(M)
display(tex,wait,but)

Type commands in the textbox, and press the button to play the animation. Toggle the slider to turn on and off the animation.


2 1 0,2 1 0,2 1 0
2 1 0,,2 1 0
2 1 0,2 1 0,2 1 0


Text(value='')

FloatSlider(value=0.0, max=5.0)

Button(description='Play', style=ButtonStyle())

A comutator : ldru rdlu dlur drul = ldrurdllurdrul \
This has order 3

Move in 2*2 box : ldru \
This has order 9

Trying a very crude BFS

In [91]:
from copy import deepcopy
M0 = [
    [[1,2],[1,2],[1,2]],
    [[1,2],None,[1,2]],
    [[1,2],[1,2],[1,2]]
]
def eq(M1,M2):
    for i in range(len(M1)):
        for j in range(len(M1[0])):
            if M1[i][j]!=M2[i][j]:
                return False
    return True
MQue = [deepcopy(M0)]
SQue = [""]
apply("ldr",MQue[0])
cur = 0
def putChildrenInQue():
    global SQue,MQue,Moves,cur
    for move in Moves:
        m = deepcopy(MQue[cur])
        if Moves[move](m):
            In = False
            for m2 in MQue:
                if eq(m,m2):
                    In = True
                    break
            if not In:
                SQue.append(SQue[cur]+move)
                MQue.append(m)
for i in range(100):
    if eq(MQue[cur],M0):
        print("found it")
        print(SQue[cur])
        print(len(SQue))
        break
    putChildrenInQue()
    cur += 1
    print(len(SQue))

4
7
8
9
11
13
15
17
19
20
21
22
23
24
25
26
found it
LUR
26


Trying to convert matrix to numbers

In [92]:
diesits = []
for l in range(6):
    for u in range(6):
        if Top(l,u)!=None:
            diesits.append((l,u))
def convertToLU(i):
    l = i//4
    u = i%4
    if u>=l-3 and l>=3:
        u += 1
    if u>=l:
        u += 1
    if u>=l+3 and l<3:
        u += 1
    return l,u
def convertToI(l,u):
    i = l*4 + u
    if u > l-3 and l>=3:
        i -= 1
    if u>=l:
        i -= 1
    if u>=l+3 and l<3:
        i -= 1
    return i
def ToN(M):
    N = 0
    for i in range(len(M)):
        for j in range(len(M[0])):
            if M[i][j]==None:
                I,J = i,j
                m = deepcopy(M)
    for i in range(len(M)):
        for j in range(len(M[0])):
            if (i!=I or j!=J):
                N = N*24 + convertToI(m[i][j][0],m[i][j][1])
    N = (24*N) + 3*I+J
    return N
def To3by3M(N):
    M = [[None]*3 for _ in range(3)]
    N,n = N//24,N%24
    I,J = n//3,n%3
    for i in range(2,-1,-1):
        for j in range(2,-1,-1):
            if i!=I or j !=J:
                M[i][j] = convertToLU(N%24)
                N=N//24
    return M


## Corrected Question

Interface and setup

In [93]:
MOVES = {
    "L":[1,3,2,4,0,5],
    "U":[2,1,3,5,4,0],
    "R":[4,0,2,1,3,5],
    "D":[5,1,0,2,4,3]
}
OFFSETS = {
    "L":(0,-1),
    "U":(-1,0),
    "R":(0,1),
    "D":(1,0)
}
X = None
M0 = [
    [0,0,0],
    [0,X,0],
    [0,0,0]
]
loc0 = (1,1)
def disp(M):
    colors = ["black"]+["gray"]*5
    S = "<table>"
    for row in M:
        S += '<tr>'
        for x in row :
            if x==None:
                S += f"<td bgcolor=white></td>"
            else:
                S += f"<td bgcolor={colors[x]}>{x}</td>"
        S += "</tr>"
    S += "</table>"
    display(HTML(S))
def apply(moves,M,loc,dt=0):
    global OFFSETS,MOVES
    for move in moves :
        i,j = loc
        I,J = OFFSETS[move]
        I += i
        J += j
        if I>=0 and I<len(M) and J>=0 and J<len(M[0]):
            M[i][j] = MOVES[move][M[I][J]]
            M[I][J] = None
            loc = (I,J)
        if dt > 0:
            clear_output(True)
            disp(M)
            sleep(dt)
    return loc
def ToN(M,loc):
    N = 0
    I,J = loc
    for i in range(len(M)):
        for j in range(len(M[0])):
            if i==I and j==J:
                continue
            N = 6*N + M[i][j]
    N = len(M)*len(M[0])*N + len(M[0])*I + J
    return N
def To3by3M(N):
    n,N = N%9,N//9
    I,J = n//3,n%3
    M=[[None]*3 for _ in range(3)]
    for i in range(2,-1,-1):
        for j in range(2,-1,-1):
            if i==I and j==J:
                continue
            M[i][j],N = N%6,N//6
    loc = (I,J)
    return M,loc
disp(M0)


0,0,0
0,,0
0,0,0


BFS

In [94]:
Loops = 0
def eq(M1,M2):
    for i in range(len(M1)):
        for j in range(len(M1[0])):
            if M1[i][j]!=M2[i][j]:
                return False
    return True
def works(M=MQue[0]):
    global M0
    return eq(M,M0)
def putChildrenInQue(MQue,LQue,SQue,visited,i=0):
    global Loops
    M = MQue[i]
    loc = LQue[i]
    S = SQue[i]
    for move in MOVES:
        m = deepcopy(M)
        newloc = apply(move,m,loc)
        if newloc == loc:
            continue
        N = ToN(m,newloc)
        if visited[N]:
            Loops += 1
            continue
        N = ToN(M,loc)
        visited[N] = True
        s = S + move
        MQue.append(m)
        LQue.append(newloc)
        SQue.append(s)
    #MQue.pop(i)
    #LQue.pop(i)
    #SQue.pop(i)
def BFS(M,loc,dt=0):
    MQue = [M]
    LQue = [loc]
    SQue = [""]
    visited = [False]*((6**8)*9)
    for i in range(10**8):
        putChildrenInQue(MQue,LQue,SQue,visited,i)
        if works(MQue[i]):
            return SQue[i]
    print("didn't work")


## playable game.

In [ ]:
M = [
    [0,0,0],
    [0,X,0],
    [0,0,0]
]
loc = (1,1)
Play = Button()
Solve = Button()
ReSet = Button()
Play.description="Play"
Solve.description = "Solve"
ReSet.description = "ReSet"
tex = Text()
wait = FloatSlider()
wait.max=5
def Reset(event):
    global M,M0,loc,loc0
    M = deepcopy(M0)
    loc = tuple(loc0)
    clear_output(True)
    disp(M)
    display(tex,wait,Play,Solve,ReSet) 
def play(event):
    global M,loc
    s = tex.value
    dt = float(wait.value)
    print(dt)
    loc = apply(s,M,loc,dt)
    clear_output(True)
    disp(M)
    display(tex,wait,Play,Solve,ReSet)    
def solve(event):
    global M,loc
    dt = float(wait.value)
    s = BFS(M,loc,dt)
    m = deepcopy(M)
    clear_output(True)
    apply(s,m,loc,dt)
    clear_output(True)
    print("Our Problem :")
    disp(M)
    print("Our solution :",s)
    display(tex,wait,Play,Solve,ReSet) 
Play.on_click(play)
Solve.on_click(solve)
ReSet.on_click(Reset)
disp(M)
display(tex,wait,Play,Solve,ReSet)

Our Problem :


4,1,0
2,,0
0,0,0


Our solution : ULDR


Text(value='LURD')

FloatSlider(value=0.7, max=5.0)

Button(description='Play', style=ButtonStyle())

Button(description='Solve', style=ButtonStyle())

Button(description='ReSet', style=ButtonStyle())